**Softmax Classification**

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [2]:
torch.manual_seed(1)

In [3]:
z = torch.FloatTensor([1,2,3])

In [4]:
hypothesis = F.softmax(z, dim = 0)
print(hypothesis)
#각 값의 총합은 1

tensor([0.0900, 0.2447, 0.6652])


In [5]:
z = torch.rand(3, 5, requires_grad=True)
hypothesis = F.softmax(z, dim = 1)

y = torch.randint(5, (3,)).long()

In [6]:
y_one_hot = torch.zeros_like(hypothesis)
print(y_one_hot)
y_one_hot.scatter_(1, y.unsqueeze(1), 1)
print(y_one_hot)

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])
tensor([[1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0.]])


In [7]:
#Low level
cost = (y_one_hot * -torch.log(hypothesis)).sum(dim=1).mean()
print(cost)

tensor(1.4689, grad_fn=<MeanBackward0>)


In [8]:
#High level1 : F.log_softmax
cost = -(y_one_hot * F.log_softmax(z, dim = 1)).sum(dim = 1).mean()
print(cost)

#High level2 : F.nll_loss
cost = F.nll_loss(F.log_softmax(z, dim = 1), y)
print(cost)

#High level3 : cross_entropy function
cost = F.cross_entropy(z, y)
print(cost)

tensor(1.4689, grad_fn=<NegBackward>)
tensor(1.4689, grad_fn=<NllLossBackward>)
tensor(1.4689, grad_fn=<NllLossBackward>)


In [9]:
x_data = [[1,2,1,1],
           [2,1,3,2],
           [3,1,3,4],
           [4,1,5,5],
           [1,7,5,5],
           [1,2,5,6],
           [1,6,6,6],
           [1,7,7,7]]
y_data = [2,2,2,1,1,1,0,0]
x_train = torch.FloatTensor(x_data)
y_train = torch.LongTensor(y_data)

In [10]:
W = torch.zeros((4, 3), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

optimizer = optim.SGD([W, b], lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

  #Cost - low level
  '''
  hypothesis = F.softmax(torch.matmul(x_train, W) + b, dim = 1)
  y_one_hot = torch.zeros_like(hypothesis)
  y_one_hot.scatter_(1, y_train.unsqueeze(1), 1)
  cost = (y_one_hot * -torch.log(hypothesis)).sum(dim = 1).mean()
  '''

  #Cost - high level
  z = torch.matmul(x_train, W) + b
  cost = F.cross_entropy(z, y_train)

  #gradient
  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  if epoch % 100 == 0:
    print('Epoch {:4d} / {}, Cost : {:.6f}'.format(
        epoch, nb_epochs, cost.item()
    ))

Epoch    0 / 1000, Cost : 1.098612
Epoch  100 / 1000, Cost : 0.761050
Epoch  200 / 1000, Cost : 0.689991
Epoch  300 / 1000, Cost : 0.643229
Epoch  400 / 1000, Cost : 0.604117
Epoch  500 / 1000, Cost : 0.568256
Epoch  600 / 1000, Cost : 0.533922
Epoch  700 / 1000, Cost : 0.500291
Epoch  800 / 1000, Cost : 0.466908
Epoch  900 / 1000, Cost : 0.433507
Epoch 1000 / 1000, Cost : 0.399962


**Class로 만들기**

In [11]:
class SoftmaxClassifierModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.linear = nn.Linear(4, 3)
  
  def forward(self, x):
    return self.linear(x)

In [12]:
model = SoftmaxClassifierModel()

optimizer = optim.SGD(model.parameters(), lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):
  
  #hypothesis
  hypothesis = model(x_train)

  #cost
  cost = F.cross_entropy(hypothesis, y_train)

  #gradient
  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  if epoch % 100 == 0:
    print('Epoch : {:4d} / {}, Cost : {:.6f}'.format(epoch, nb_epochs, cost.item()))

Epoch :    0 / 1000, Cost : 1.849513
Epoch :  100 / 1000, Cost : 0.689894
Epoch :  200 / 1000, Cost : 0.609259
Epoch :  300 / 1000, Cost : 0.551218
Epoch :  400 / 1000, Cost : 0.500141
Epoch :  500 / 1000, Cost : 0.451947
Epoch :  600 / 1000, Cost : 0.405051
Epoch :  700 / 1000, Cost : 0.358733
Epoch :  800 / 1000, Cost : 0.312912
Epoch :  900 / 1000, Cost : 0.269522
Epoch : 1000 / 1000, Cost : 0.241922


**test로 검증해보기**

In [13]:
x_train = torch.FloatTensor([[1,2,1],
                            [1,3,2],
                            [1,3,4],
                            [1,5,5],
                            [1,7,5],
                            [1,2,5],
                            [1,6,6],
                            [1,7,7]])
y_train = torch.LongTensor([2,2,2,1,1,1,0,0])

x_test = torch.FloatTensor([[2,1,1],[3,1,2],[3,3,4]])
y_test = torch.LongTensor([2,2,2])

In [14]:
class SoftmaxClassifierModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.linear = nn.Linear(3, 3)
  
  def forward(self, x):
    return self.linear(x)

In [15]:
model = SoftmaxClassifierModel()

optimizer = optim.SGD(model.parameters(), lr=0.1)

In [16]:
def train(model, optimizer, x_train, y_train):
  nb_epochs = 20
  for epoch in range(nb_epochs + 1):

    prediction = model(x_train)

    cost = F.cross_entropy(prediction, y_train)

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    print('Epoch {:4d} / {}, Cost : {:.6f}'.format(epoch, nb_epochs, cost.item()))

In [17]:
def test(model, optimizer, x_test, y_test):
  prediction = model(x_test)
  predicted_classes = prediction.max(1)[1]
  correct_count = (predicted_classes == y_test).sum().item()
  cost = F.cross_entropy(prediction, y_test)

  print('Accuracy: {}% Cost: {:.6f}'.format(correct_count / len(y_test) * 100, cost.item()))

In [18]:
train(model, optimizer, x_train, y_train)

Epoch    0 / 20, Cost : 2.558566
Epoch    1 / 20, Cost : 1.796931
Epoch    2 / 20, Cost : 1.240436
Epoch    3 / 20, Cost : 1.149199
Epoch    4 / 20, Cost : 1.136782
Epoch    5 / 20, Cost : 1.127878
Epoch    6 / 20, Cost : 1.119454
Epoch    7 / 20, Cost : 1.111317
Epoch    8 / 20, Cost : 1.103407
Epoch    9 / 20, Cost : 1.095682
Epoch   10 / 20, Cost : 1.088116
Epoch   11 / 20, Cost : 1.080694
Epoch   12 / 20, Cost : 1.073404
Epoch   13 / 20, Cost : 1.066240
Epoch   14 / 20, Cost : 1.059197
Epoch   15 / 20, Cost : 1.052270
Epoch   16 / 20, Cost : 1.045456
Epoch   17 / 20, Cost : 1.038752
Epoch   18 / 20, Cost : 1.032158
Epoch   19 / 20, Cost : 1.025669
Epoch   20 / 20, Cost : 1.019286


In [19]:
test(model, optimizer, x_test, y_test)
#->  오버피팅된 상태 

Accuracy: 0.0% Cost: 1.773133


In [20]:
model = SoftmaxClassifierModel()
#학습률에 따라서 cost의 변화를 본다.
#너무 크면 cost가 늘어나고 너무 작으면 줄어들지 않는다.
optimizer = optim.SGD(model.parameters(), lr = 0.1)

train(model, optimizer, x_train, y_train)

Epoch    0 / 20, Cost : 2.180209
Epoch    1 / 20, Cost : 1.414980
Epoch    2 / 20, Cost : 1.316441
Epoch    3 / 20, Cost : 1.272446
Epoch    4 / 20, Cost : 1.250457
Epoch    5 / 20, Cost : 1.233646
Epoch    6 / 20, Cost : 1.218635
Epoch    7 / 20, Cost : 1.204813
Epoch    8 / 20, Cost : 1.191737
Epoch    9 / 20, Cost : 1.179248
Epoch   10 / 20, Cost : 1.167246
Epoch   11 / 20, Cost : 1.155676
Epoch   12 / 20, Cost : 1.144503
Epoch   13 / 20, Cost : 1.133701
Epoch   14 / 20, Cost : 1.123248
Epoch   15 / 20, Cost : 1.113127
Epoch   16 / 20, Cost : 1.103322
Epoch   17 / 20, Cost : 1.093816
Epoch   18 / 20, Cost : 1.084598
Epoch   19 / 20, Cost : 1.075652
Epoch   20 / 20, Cost : 1.066968


**데이터 전처리**

In [21]:
x_train = torch.FloatTensor([[73,80,75],[93, 88, 93], [89, 91, 90], [96, 98, 100], [73, 66, 70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

In [22]:
mu = x_train.mean(dim = 0)
sigma = x_train.std(dim = 0)
norm_x_train = (x_train - mu) / sigma

print(norm_x_train)

tensor([[-1.0674, -0.3758, -0.8398],
        [ 0.7418,  0.2778,  0.5863],
        [ 0.3799,  0.5229,  0.3486],
        [ 1.0132,  1.0948,  1.1409],
        [-1.0674, -1.5197, -1.2360]])


In [23]:
class MultivariateLinearRegressionModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.linear = nn.Linear(3, 1)

  def forward(self, x):
    return self.linear(x)

model = MultivariateLinearRegressionModel()

optimizer = optim.SGD(model.parameters(), lr = 1e-1)

In [24]:
def train(model, optimizer, x_train, y_train):
  nb_epochs = 20
  for epoch in range(nb_epochs + 1):

    prediction = model(x_train)

    cost = F.mse_loss(prediction, y_train)

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    print('Epoch {:4d} / {}, Cost : {:.6f}'.format(epoch, nb_epochs, cost.item()))

In [25]:
train(model, optimizer, norm_x_train, y_train)

Epoch    0 / 20, Cost : 29784.714844
Epoch    1 / 20, Cost : 18917.908203
Epoch    2 / 20, Cost : 12065.722656
Epoch    3 / 20, Cost : 7710.173340
Epoch    4 / 20, Cost : 4931.301270
Epoch    5 / 20, Cost : 3155.339600
Epoch    6 / 20, Cost : 2019.448608
Epoch    7 / 20, Cost : 1292.681885
Epoch    8 / 20, Cost : 827.603149
Epoch    9 / 20, Cost : 529.960083
Epoch   10 / 20, Cost : 339.463928
Epoch   11 / 20, Cost : 217.538574
Epoch   12 / 20, Cost : 139.497437
Epoch   13 / 20, Cost : 89.542427
Epoch   14 / 20, Cost : 57.562733
Epoch   15 / 20, Cost : 37.087402
Epoch   16 / 20, Cost : 23.975254
Epoch   17 / 20, Cost : 15.575839
Epoch   18 / 20, Cost : 10.192981
Epoch   19 / 20, Cost : 6.740880
Epoch   20 / 20, Cost : 4.524698
